In [1]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup

r = requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

In [2]:
raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
len(card_list) # len should be lower than total (duplicate basic lands)

178

In [3]:
cached_data = 'muldrotha.csv'
def get_card_data(card_name):
    r = requests.get('https://api.scryfall.com/cards/search/?q=' + card_name)
    return(r.json()) # we _should_ always get the card we need since titles are exact
card_data = []
if not os.path.exists(cached_data):
    pool = mp.Pool()
    card_data = pool.map(get_card_data, card_list)
    pool.close()

In [4]:
card_df = None
if os.path.exists(cached_data):
    card_df =pd.read_csv('muldrotha.csv')
else:
    card_df = pd.DataFrame(list(map(lambda x: x['data'][0], card_data)))
    card_df = card_df[['name','type_line','cmc','mana_cost','oracle_text','power','toughness','usd']]
    card_df.to_csv('muldrotha.csv', sep = ',', encoding = 'utf-8', index = False)
card_df.head()

,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
0,"Muldrotha, the Gravetide",Legendary Creature — Elemental Avatar,6.0,{3}{U}{B}{G},"During each of your turns, you may play up to ...",6,6,6.49
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or a...,NaN,NaN,4.69
2,Acidic Slime,Creature — Ooze,5.0,{3}{G}{G},Deathtouch (Any amount of damage this deals to...,2,2,0.25
3,Alchemist's Refuge,Land,0.0,,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: ...",NaN,NaN,2.06
4,Arcane Lighthouse,Land,0.0,,"{T}: Add {C} to your mana pool.\n{1}, {T}: Unt...",NaN,NaN,2.79


In [5]:
lands = card_df[card_df['type_line'].str.contains('Land', na = False)]
print('Land count: ' + str(len(lands)))
lands

Land count: 49


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
3,Alchemist's Refuge,Land,0.0,,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: ...",NaN,NaN,2.06
4,Arcane Lighthouse,Land,0.0,,"{T}: Add {C} to your mana pool.\n{1}, {T}: Unt...",NaN,NaN,2.79
6,Ash Barrens,Land,0.0,,{T}: Add {C} to your mana pool.\nBasic landcyc...,NaN,NaN,2.18
8,Bad River,Land,0.0,,"Bad River enters the battlefield tapped.\n{T},...",NaN,NaN,NaN
13,Blooming Marsh,Land,0.0,,Blooming Marsh enters the battlefield tapped u...,NaN,NaN,5.17
15,Botanical Sanctum,Land,0.0,,Botanical Sanctum enters the battlefield tappe...,NaN,NaN,6.31
18,Breeding Pool,Land — Forest Island,0.0,,({T}: Add {G} or {U} to your mana pool.)\nAs B...,NaN,NaN,14.53
19,Cabal Coffers,Land,0.0,,"{2}, {T}: Add {B} to your mana pool for each S...",NaN,NaN,29.22
24,Choked Estuary,Land,0.0,,"As Choked Estuary enters the battlefield, you ...",NaN,NaN,0.99
25,Command Beacon,Land,0.0,,"{T}: Add {C} to your mana pool.\n{T}, Sacrific...",NaN,NaN,16.29


In [6]:
mana_rocks = card_df[card_df['oracle_text'].str.contains('mana pool|Mana pool', na = False)]
mana_rocks = mana_rocks[-mana_rocks['type_line'].str.contains('Land|land', na = False)]
print('Mana ramp count: ' + str(len(mana_rocks)))
mana_rocks

Mana ramp count: 21


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
7,Awakening Zone,Enchantment,3.0,{2}{G},"At the beginning of your upkeep, you may creat...",NaN,NaN,3.08
11,Birds of Paradise,Creature — Bird,1.0,{G},Flying\n{T}: Add one mana of any color to your...,0,1,6.95
12,Bloom Tender,Creature — Elf Druid,2.0,{1}{G},{T}: For each color among permanents you contr...,1,1,29.30
21,Catacomb Sifter,Creature — Eldrazi Drone,3.0,{1}{B}{G},Devoid (This card has no color.)\nWhen Catacom...,2,3,0.09
27,Commander's Sphere,Artifact,3.0,{3},{T}: Add to your mana pool one mana of any col...,NaN,NaN,0.25
34,Cryptolith Rite,Enchantment,2.0,{1}{G},"Creatures you control have ""{T}: Add one mana ...",NaN,NaN,2.72
35,Crystalline Crawler,Artifact Creature — Construct,4.0,{4},Converge — Crystalline Crawler enters the batt...,1,1,4.99
43,Deathrite Shaman,Creature — Elf Shaman,1.0,{B/G},{T}: Exile target land card from a graveyard. ...,1,2,4.40
49,Dimir Cluestone,Artifact,3.0,{3},{T}: Add {U} or {B} to your mana pool.\n{U}{B}...,NaN,NaN,0.08
51,Dimir Signet,Artifact,2.0,{2},"{1}, {T}: Add {U}{B} to your mana pool.",NaN,NaN,1.49


In [7]:
card_draw = card_df[card_df['oracle_text'].str.contains('Draw|draw', na = False)]
print('Card draw count: ' + str(len(card_draw)))
card_draw

Card draw count: 37


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
16,Braingeyser,Sorcery,2.0,{X}{U}{U},Target player draws X cards.,NaN,NaN,NaN
17,Brainstorm,Instant,1.0,{U},"Draw three cards, then put two cards from your...",NaN,NaN,0.61
20,Careful Study,Sorcery,1.0,{U},"Draw two cards, then discard two cards.",NaN,NaN,0.59
23,Chart a Course,Sorcery,2.0,{1}{U},Draw two cards. Then discard a card unless you...,NaN,NaN,0.44
27,Commander's Sphere,Artifact,3.0,{3},{T}: Add to your mana pool one mana of any col...,NaN,NaN,0.25
28,Consecrated Sphinx,Creature — Sphinx,6.0,{4}{U}{U},"Flying\nWhenever an opponent draws a card, you...",4,6,13.56
45,Desert of the Glorified,Land — Desert,0.0,,Desert of the Glorified enters the battlefield...,NaN,NaN,0.10
46,Desert of the Indomitable,Land — Desert,0.0,,Desert of the Indomitable enters the battlefie...,NaN,NaN,0.09
47,Desert of the Mindful,Land — Desert,0.0,,Desert of the Mindful enters the battlefield t...,NaN,NaN,0.10
49,Dimir Cluestone,Artifact,3.0,{3},{T}: Add {U} or {B} to your mana pool.\n{U}{B}...,NaN,NaN,0.08
